# EMODNET human activities WFS

## Install packages

In [1]:
packages = ['owslib',
            'geopandas',
            'matplotlib.pyplot',
            'contextily',
            'folium'
           ]
for package in packages:
    !pip install {package} > /dev/null 2>&1
for package in packages:
    exec(f'import {package}')
from owslib.wfs import WebFeatureService
import geopandas as gpd
import contextily as ctx
import matplotlib.pyplot as plt
from folium.plugins import MarkerCluster

## Connect to WFS

In [2]:
WFS_URL = "https://ows.emodnet-humanactivities.eu/wfs"
wfs = WebFeatureService(url=WFS_URL, version="2.0.0")

## List all layers in this WFS

In [3]:
for i, layer in enumerate(wfs.contents):
    info = wfs.contents[layer]
    print(f"{i} / {len(wfs.contents)}, {layer} — title: {info.title}")

0 / 134, emodnet:activelicenses — title: Active Licences
1 / 134, emodnet:advisorycouncils — title: Advisory Councils
2 / 134, emodnet:aquaculture — title: Advisory Councils (Aquaculture)
3 / 134, emodnet:baltic — title: Advisory Councils (Baltic)
4 / 134, emodnet:blacksea — title: Advisory Councils (Black Sea)
5 / 134, emodnet:longdistancefleet — title: Advisory Councils (Long Distance Fleet)
6 / 134, emodnet:market — title: Advisory Councils (Market)
7 / 134, emodnet:mediterranean — title: Advisory Councils (Mediterranean)
8 / 134, emodnet:northsea — title: Advisory Councils (North Sea)
9 / 134, emodnet:northwesternwaters — title: Advisory Councils (North Western Waters)
10 / 134, emodnet:outermostregions — title: Advisory Councils (Outermost Regions)
11 / 134, emodnet:pelagicstocks — title: Advisory Councils (Pelagic Stocks)
12 / 134, emodnet:southwesternwaters — title: Advisory Councils (Southwestern Waters)
13 / 134, emodnet:aggregateareas — title: Aggregate Extraction Areas
14 / 

## Fetch layer windfarm polygons

In [4]:
LATER_TO_FETCH = "emodnet:windfarmspoly"

response = wfs.getfeature(
    typename=LATER_TO_FETCH,
    outputFormat="application/json"  # ask for GeoJSON directly
    )
gdf = gpd.read_file(response)

### View the data

In [5]:
print(gdf.head())

                  id         country                                  name  \
0  windfarmspoly.241  United Kingdom             Dogger Bank East Met Mast   
1   windfarmspoly.15  United Kingdom            Scarweather Sands Met Mast   
2   windfarmspoly.16  United Kingdom  Shell Flats No. 1 (Western) Met Mast   
3    windfarmspoly.2  United Kingdom                       Islay Demo Zone   
4   windfarmspoly.17  United Kingdom  Shell Flats No. 2 (Eastern) Met Mast   

   n_turbines  power_mw      status updateyear  year     dist_coast  \
0         1.0       NaN  Dismantled       None  None  210099.226384   
1         1.0       NaN  Dismantled       None  None    8166.582792   
2         1.0       NaN  Production       None  None   15769.864456   
3        10.0      10.0    Approved       None  None    1122.942248   
4         1.0       NaN  Production       None  None   10035.848868   

   area_sqkm notes  shape_leng  shape_area  \
0   0.195959  None    0.019699    0.000028   
1   0.031385

## Plot as interactive map

In [6]:
import folium

# Keep only Polygon / MultiPolygon geometries
gdf_poly = gdf[gdf.geometry.type.isin(["Polygon", "MultiPolygon"])].copy()

# Keep only serializable columns (e.g., object, str, int, float)
serializable_cols = gdf_poly.select_dtypes(include=['int','float','object','bool']).columns
gdf_poly = gdf_poly[serializable_cols.tolist() + ['geometry']]

# Ensure CRS is EPSG:4326
if gdf_poly.crs is None:
    gdf_poly = gdf_poly.set_crs("EPSG:4326")
elif gdf_poly.crs.to_epsg() != 4326:
    gdf_poly = gdf_poly.to_crs(epsg=4326)

# Create folium map centered on the mean bounds
bounds = gdf_poly.total_bounds  # [minx, miny, maxx, maxy]
center = [(bounds[1] + bounds[3])/2, (bounds[0] + bounds[2])/2]
m = folium.Map(location=center, zoom_start=5, tiles="OpenStreetMap")

# Add polygons as GeoJSON
folium.GeoJson(
    gdf_poly.__geo_interface__,
    name="Windfarms",
    tooltip=folium.GeoJsonTooltip(
        fields=[col for col in gdf_poly.columns if col != 'geometry'],
        aliases=[col + ":" for col in gdf_poly.columns if col != 'geometry'],
    ),
    style_function=lambda feature: {
        'fillColor': 'red',
        'color': 'darkred',
        'weight': 1,
        'fillOpacity': 0.5
    }
).add_to(m)

# Add layer control
folium.LayerControl().add_to(m)

m

